# Ler pacotes

In [41]:
import os

In [42]:
current_path = os.getcwd()
if current_path.split('/')[-1] == 'content':
  %pip install trajminer -q

  from google.colab import drive

  drive.mount('/content/drive')
  print(f"current path:{current_path}. Moving to the project path.")
  os.chdir("drive/MyDrive/Colab Notebooks/mat_tree/")
  print(f"Current path: {os.getcwd()}")
else:
  print("Working locally.")
  print(f"current path:{current_path}.")

Working locally.
current path:g:\Meu Drive\Colab Notebooks\mat_tree\preprocessing.


In [43]:
import pandas as pd
import numpy as np
import plotly.express as px

# import trajminer as tm
# from trajminer.similarity import MSM, MUITAS
# from trajminer.utils.distance import discrete, euclidean
# from tqdm import tqdm
# from multiprocessing import Pool, cpu_count

# from scipy.spatial.distance import euclidean

# Ler dataset

In [99]:
dtypes = {'UF':'string', 'voto':'string', 'orientacaoGoverno':'string', 'anoProposicao':'int32',
          'anoVotacao':'int32', 'tipoProposicao':'string', 'governo':'string', 'parlamentar':'string',
        #   'data':'datetime64[ns]',
           'idVotacao':'string', 'tid':'int64', 'label':'string', 'horaVotacao':'int32', 'diaDaSemanaVotacao':'int32', 'diaDaSemanaVotacaoNome':'string', 'diaDoAno':'int32', 'diaDoMesVotacao':'int32', 'mesVotacao': 'int32', 'delayVotacaoAnos':'int32', 'alinhamento':'string'}
dataset_path = "../datasets/basometro/"
basometro_df = pd.read_csv(dataset_path+'basometro.csv', encoding='latin-1', dtype=dtypes, parse_dates=['data'])

In [45]:
basometro_df.head(3)

,UF,voto,orientacaoGoverno,anoProposicao,anoVotacao,tipoProposicao,governo,parlamentar,data,idVotacao,tid,label,horaVotacao,diaDaSemanaVotacao,diaDaSemanaVotacaoNome,diaDoAno,diaDoMesVotacao,mesVotacao,delayVotacaoAnos,alinhamento
0,PR,Sim,Sim,2002,2003,MSC,Lula 1,Abelardo Lupion,2003-02-25 18:54:00,25-2-2003.18.54.736,5318,DEM,18,1,Tuesday,56,25,2,1,votou com o governo
1,PR,Sim,Sim,2002,2003,MPV,Lula 1,Abelardo Lupion,2003-03-18 19:30:00,18-3-2003.19.30.742,5318,DEM,19,1,Tuesday,77,18,3,1,votou com o governo
2,PR,Sim,Não,1996,2003,PL,Lula 1,Abelardo Lupion,2003-03-19 18:37:00,19-3-2003.18.37.743,5318,DEM,18,2,Wednesday,78,19,3,7,votou contra o governo


In [46]:
# basometro_df.info()

# Preprocessing

## Métodos auxiliares

In [47]:
def convert_idVotacao(x):
    """Método para extrair o ano presente no idVotacao"""
    try:
        return int(x['idVotacao'].split(".")[0].split("-")[-1])
    except:
        return int(x['idVotacao'].split(".")[-1])

In [48]:
def get_periodo_mandato(ano_votacao):
    """Método para saber o período de mandado eleito com base no ano da votação."""
    if (ano_votacao >= 2003) and (ano_votacao <= 2006):
        return "2003-2006"
    elif (ano_votacao >= 2007) and (ano_votacao <= 2010):
        return "2007-2010"
    elif (ano_votacao >= 2011) and (ano_votacao <= 2014):
        return "2011-2014"
    elif (ano_votacao >= 2015) and (ano_votacao <= 2018):
        return "2015-2018"
    elif (ano_votacao >= 2019) and (ano_votacao <= 2022):
        return "2019-2022"
    else:
        return "outros"

## Feature engineering: alinhamento intra partidário

### Calcular a quantidade total de deputados por partido no período de mandado da presidencia.

In [49]:
tmp = basometro_df[['label','anoVotacao','parlamentar']].copy()
tmp['periodo_mandato'] = tmp.apply(lambda x: get_periodo_mandato(x['anoVotacao']),axis=1)
tmp = tmp.sort_values(by=['label','anoVotacao'])
# display(tmp.head(3))

### Calculando o atributo qtd_parlamentares_partido.
### Com a agregação abaixo nós queremos saber quantos parlamentares eleitos cada partido tinha em cada período de mandato.
df_tamanho_partidos = tmp.drop_duplicates(subset=['label','parlamentar','periodo_mandato'])[['label','periodo_mandato','parlamentar']] \
   .groupby(by=['label','periodo_mandato']).agg(qtd_parlamentares_partido = ('parlamentar','count')).reset_index()
display(df_tamanho_partidos.head())

,label,periodo_mandato,qtd_parlamentares_partido
0,Avante,2007-2010,1
1,Avante,2011-2014,2
2,Avante,2015-2018,7
3,Avante,2019-2022,8
4,Cidadania,2003-2006,33


In [50]:
# del tmp

### transformar df em dict para acelerar consulta
dict_tamanho_partidos = {}
for i, row in df_tamanho_partidos.iterrows():
    if row['label'] in dict_tamanho_partidos:
        dict_tamanho_partidos[row['label']][row['periodo_mandato']] = row['qtd_parlamentares_partido']
    else:
        dict_tamanho_partidos.update({row['label']:{}})
        dict_tamanho_partidos[row['label']][row['periodo_mandato']] = row['qtd_parlamentares_partido']
# del df_tamanho_partidos

### Calcular a quantidade de votos por Partido, idVotacao e Proposição.

O tipo de voto pode ser: Sim, Não, Obstrução, Abstenção e Ausente.

Problema de duplicatas. Olhar seção 'Problemas' para detalhes.

In [51]:

df_clean_tmp = basometro_df.drop_duplicates(subset=['idVotacao','parlamentar'])

tmp_gp = df_clean_tmp.groupby(['label','idVotacao','tipoProposicao','voto']).agg(count = ('voto','count')) \
         .reset_index() \
         .sort_values(by=['label','idVotacao','tipoProposicao'])

print(tmp_gp.shape)
tmp_gp.head(10)

(99075, 5)


,label,idVotacao,tipoProposicao,voto,count
0,Avante,1-10-2019.00.20.16943,PL,Não,2
1,Avante,1-12-2021.17.21.17189,PL,Sim,1
2,Avante,1-12-2021.17.39.17189,PL,Sim,1
3,Avante,1-12-2021.17.54.17189,PL,Sim,1
4,Avante,1-4-2009.16.30.3673,MPV,Não,1
5,Avante,1-4-2009.19.23.3673,MPV,Sim,1
6,Avante,1-7-2021.12.04.17120,PL,Sim,1
7,Avante,1-7-2021.12.24.17120,PL,Sim,1
8,Avante,1-7-2021.13.22.17120,PL,Sim,2
9,Avante,1-8-2017.18.14.16274,MPV,Não,1


Pivotar o resultado do groupby para em seguida calcular as proporções de votos Sim e Não do partido.

In [52]:
table = pd.pivot_table(tmp_gp, values=['count'], index=['label','idVotacao','tipoProposicao'], columns=['voto'],
                       aggfunc='sum', fill_value=0).reset_index().reset_index(drop=True).values

table = pd.DataFrame(table, columns=['label','idVotacao','tipoProposicao','Abstenção','Obstrução','Não','AUSENTE','Sim'])
table['ano'] = table.apply(lambda x: convert_idVotacao(x),axis=1)
reorder_cols = ['label','ano','idVotacao','tipoProposicao','Abstenção','Obstrução','AUSENTE','Não','Sim']
table = table[reorder_cols].sort_values(by=['label','ano'])

print(table.shape)
display(table.head(5))
display(table[table['label']=='Avante'].tail(5))

(69913, 9)


,label,ano,idVotacao,tipoProposicao,Abstenção,Obstrução,AUSENTE,Não,Sim
4,Avante,2009,1-4-2009.16.30.3673,MPV,0,0,0,1,0
5,Avante,2009,1-4-2009.19.23.3673,MPV,0,0,0,0,1
10,Avante,2009,1-9-2009.17.17.3812,MPV,0,0,0,1,0
37,Avante,2009,10-11-2009.18.25.3883,MPV,0,0,0,1,0
38,Avante,2009,10-11-2009.19.54.3883,MPV,0,0,0,1,0


,label,ano,idVotacao,tipoProposicao,Abstenção,Obstrução,AUSENTE,Não,Sim
1934,Avante,2022,9-2-2022.20.50.17206,PL,0,0,0,4,0
1935,Avante,2022,9-2-2022.21.35.17206,MPV,0,0,0,3,0
1936,Avante,2022,9-3-2022.21.04.17215,REQ,0,0,0,0,4
1937,Avante,2022,9-3-2022.22.02.17215,PEC,0,0,0,0,4
1938,Avante,2022,9-3-2022.22.11.17215,PEC,0,0,0,0,4


In [53]:
# del tmp_gp

Inserindo atributos para uma nova tabela

In [70]:
lambda_ = lambda x: get_periodo_mandato(convert_idVotacao(x))
table['periodo_mandato'] = table.apply(lambda_, axis=1)
table['qtd_parlamentares_partido'] = table.apply(lambda x: dict_tamanho_partidos[x['label']][x['periodo_mandato']], axis=1)

lambda_ = lambda x: x['qtd_parlamentares_partido']-(x['Abstenção']+x['Obstrução']+x['Não']+x['Sim'])
table['ausente_calculado'] = table.apply(lambda_, axis=1)

table['total_votos'] = table.apply(lambda x: x['Abstenção']+x['Obstrução']+x['Não']+x['Sim'], axis=1)

lambda_ = lambda x: np.round((x['Sim'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_sim_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['Não'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_nao_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['Abstenção'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_abstencao_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['Obstrução'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_obstrucao_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['ausente_calculado'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_ausente_%'] = table.apply(lambda_ ,axis=1)

reorder_cols = ['label','idVotacao','ano','periodo_mandato','tipoProposicao','Abstenção','Obstrução','AUSENTE',
                'ausente_calculado','Não', 'Sim','total_votos','qtd_parlamentares_partido','partido_voto_sim_%','partido_voto_nao_%', 'partido_voto_abstencao_%', 'partido_voto_obstrucao_%', 'partido_voto_ausente_%']

new_table = table[reorder_cols].copy()
# new_table.head()
new_table.tail()

,label,idVotacao,ano,periodo_mandato,tipoProposicao,Abstenção,Obstrução,AUSENTE,ausente_calculado,Não,Sim,total_votos,qtd_parlamentares_partido,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%
69908,União,7-4-2022.13.20.17230,2022,2019-2022,PL,0,0,0,55,29,2,31,86,2.33,33.72,0.0,0.00,63.95
69909,União,8-3-2022.20.26.17214,2022,2019-2022,PLP,0,0,0,30,15,41,56,86,47.67,17.44,0.0,0.00,34.88
69910,União,9-3-2022.21.04.17215,2022,2019-2022,REQ,0,1,0,26,7,52,60,86,60.47,8.14,0.0,1.16,30.23
69911,União,9-3-2022.22.02.17215,2022,2019-2022,PEC,0,1,0,36,2,47,50,86,54.65,2.33,0.0,1.16,41.86
69912,União,9-3-2022.22.11.17215,2022,2019-2022,PEC,0,1,0,29,1,55,57,86,63.95,1.16,0.0,1.16,33.72


In [71]:
drop_cols = ['label', 'idVotacao', 'ano', 'periodo_mandato', 'tipoProposicao', 'Abstenção', 'Obstrução', 'AUSENTE',
             'ausente_calculado', 'Não', 'Sim']
new_table.drop(columns=drop_cols).sort_values(by=['total_votos'],ascending=False).head()

,total_votos,qtd_parlamentares_partido,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%
51003,92,107,85.98,0.00,0.0,0.00,14.02
51004,92,107,83.18,2.80,0.0,0.00,14.02
50837,91,107,74.77,2.80,0.0,7.48,14.95
51006,91,107,0.00,82.24,0.0,2.80,14.95
50193,90,107,74.77,3.74,0.0,5.61,15.89


Inferimos a orientação do partido sobre uma votação sendo como maior proporção de votos que a mesma recebeu. Por exemplo, se uma Proposição teve a maior proporção de votos SIM, então a orientação do partido é inferido como votar SIM.

In [92]:
def get_orientacao_partido_voto(x):
    votos = ['partido_voto_sim_%', 'partido_voto_nao_%', 'partido_voto_abstencao_%', 'partido_voto_obstrucao_%',
                   'partido_voto_ausente_%']
    map_name = {'sim':'Sim', 'nao':'Não', 'abstencao':'Abstenção', 'obstrucao':'Obstrução', 'ausente':'Ausente'}
    max_proporcao  = 0
    max_voto = ''

    for voto in votos:
        if x[voto] > max_proporcao:
            max_voto = voto.split('_')[-2]
            max_proporcao = x[voto]
    
    return map_name[max_voto]

In [93]:
new_table['orientacao_partido_voto'] = new_table.apply(lambda x: get_orientacao_partido_voto(x), axis=1)

new_table.drop(columns=drop_cols).tail()

,total_votos,qtd_parlamentares_partido,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%,orientacao_partido_voto
69908,31,86,2.33,33.72,0.0,0.00,63.95,Ausente
69909,56,86,47.67,17.44,0.0,0.00,34.88,Sim
69910,60,86,60.47,8.14,0.0,1.16,30.23,Sim
69911,50,86,54.65,2.33,0.0,1.16,41.86,Sim
69912,57,86,63.95,1.16,0.0,1.16,33.72,Sim


In [94]:
### criando dict do table para preencher o dataset do basomentro com as novas infos
info_orientacao_partido = {}
cols_votos = ['partido_voto_sim_%','partido_voto_nao_%','partido_voto_abstencao_%','partido_voto_obstrucao_%',
              'partido_voto_ausente_%','orientacao_partido_voto']

for index, row in new_table.iterrows():
    if row['label'] not in info_orientacao_partido.keys():
        info_orientacao_partido.update({row['label']:{}})
    
    if row['idVotacao'] in info_orientacao_partido[row['label']]:
        if row['tipoProposicao'] in info_orientacao_partido[row['label']][row['idVotacao']]:
            for col in cols_votos:
                if col not in info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']]:
                    info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']] = {col: row[col]}
        else:# acho que nao precisa
            info_orientacao_partido[row['label']][row['idVotacao']] = {row['tipoProposicao']:{}}
            for col in cols_votos:
                if col not in info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']]:
                    info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']][col] = row[col]
    else:
        info_orientacao_partido[row['label']][row['idVotacao']] = {}
        info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']] = {}

        for col in cols_votos:
                if col not in info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']]:
                    info_orientacao_partido[row['label']][row['idVotacao']][row['tipoProposicao']][col] = row[col]

# del table

In [95]:
# info_orientacao_partido
info_orientacao_partido['União']['7-4-2022.13.20.17230']

{'PL': {'partido_voto_sim_%': 2.33,
  'partido_voto_nao_%': 33.72,
  'partido_voto_abstencao_%': 0.0,
  'partido_voto_obstrucao_%': 0.0,
  'partido_voto_ausente_%': 63.95,
  'orientacao_partido_voto': 'Ausente'}}

## Preencher dados no dataset

In [96]:
def get_tipoVotoPartido(x, tipoVotoPartido):
    return info_orientacao_partido[x['label']][x['idVotacao']][x['tipoProposicao']][tipoVotoPartido]

In [89]:
basometro_df['voto'].unique()

<StringArray>
['Sim', 'Não', 'Obstrução', 'Abstenção', 'AUSENTE']
Length: 5, dtype: string

In [100]:
print("Inserindo atributos sobre proporção de vatos e orientação partido.")
for tipoVoto in cols_votos:
    print(f"\t Tipo voto: '{tipoVoto}'...",end='')
    lambda_ = lambda x: get_tipoVotoPartido(x, tipoVoto)
    basometro_df[tipoVoto] = basometro_df.apply(lambda_, axis=1)
    print("DONE")

print("Criando atributo 'alinhamento_candidato'...",end='')
lambda_ = lambda x: 'votou com o partido' if x['orientacao_partido_voto'].lower() == x['voto'].lower() else 'votou contra o partido'
basometro_df['alinhamento_candidato'] = basometro_df.apply(lambda_, axis=1)
print("DONE")

basometro_df.head()

Inserindo atributos sobre proporção de vatos e orientação partido.
	 Tipo voto: 'partido_voto_sim_%'...DONE
	 Tipo voto: 'partido_voto_nao_%'...DONE
	 Tipo voto: 'partido_voto_abstencao_%'...DONE
	 Tipo voto: 'partido_voto_obstrucao_%'...DONE
	 Tipo voto: 'partido_voto_ausente_%'...DONE
	 Tipo voto: 'orientacao_partido_voto'...DONE
Criando atributo 'alinhamento_candidato'...DONE


,UF,voto,orientacaoGoverno,anoProposicao,anoVotacao,tipoProposicao,governo,parlamentar,data,idVotacao,...,mesVotacao,delayVotacaoAnos,alinhamento,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%,orientacao_partido_voto,alinhamento_candidato
0,PR,Sim,Sim,2002,2003,MSC,Lula 1,Abelardo Lupion,2003-02-25 18:54:00,25-2-2003.18.54.736,...,2,1,votou com o governo,55.67,0.00,0.0,0.0,44.33,Sim,votou com o partido
1,PR,Sim,Sim,2002,2003,MPV,Lula 1,Abelardo Lupion,2003-03-18 19:30:00,18-3-2003.19.30.742,...,3,1,votou com o governo,52.58,13.40,0.0,0.0,34.02,Sim,votou com o partido
2,PR,Sim,Não,1996,2003,PL,Lula 1,Abelardo Lupion,2003-03-19 18:37:00,19-3-2003.18.37.743,...,3,7,votou contra o governo,53.61,4.12,0.0,0.0,42.27,Sim,votou com o partido
3,PR,Sim,Não,2002,2003,MPV,Lula 1,Abelardo Lupion,2003-03-26 19:16:00,26-3-2003.19.16.746,...,3,1,votou contra o governo,59.79,5.15,0.0,0.0,35.05,Sim,votou com o partido
4,PR,Sim,Não,2003,2003,MPV,Lula 1,Abelardo Lupion,2003-08-04 18:02:00,8-4-2003.18.02.754,...,4,0,votou contra o governo,55.67,6.19,0.0,0.0,38.14,Sim,votou com o partido


In [101]:
print(f"basometro no-clean shape: {basometro_df.shape}")
print(f"basometro clean shape: {basometro_df.drop_duplicates(subset=['label','idVotacao','parlamentar']).shape}")
print("Salvando dataset com novas features...",end='')
basometro_df.drop_duplicates(subset=['label','idVotacao','parlamentar']).to_csv(dataset_path+'basometro_v2.csv', encoding='latin-1', index=False)
print("DONE")

basometro no-clean shape: (1048575, 27)
basometro clean shape: (1019364, 27)
Salvando dataset com novas features...DONE


# Problemas no dataset

Investigar a quantidade total de votos maior que a quantidade de parlamentares.

Corrigir o cálculo das novas features.

In [20]:
display(df_tamanho_partidos.head())

,label,periodo_mandato,qtd_parlamentares_partido
0,Avante,2007-2010,1
1,Avante,2011-2014,2
2,Avante,2015-2018,7
3,Avante,2019-2022,8
4,Cidadania,2003-2006,33


In [21]:
### CHECANDO INSTÂNCIA
mask= (basometro_df['label']=='Avante') & (basometro_df['anoVotacao'].between(2007,2010))
display(basometro_df[mask][['label','anoVotacao','data','idVotacao','tipoProposicao','parlamentar']] \
    .sort_values(by=['data'],ascending=True)\
    .head(5))

### CHECANDO INSTÂNCIA
display(tmp[(tmp['periodo_mandato']=='2007-2010') & (tmp['label']=='Avante')].drop_duplicates(subset=['parlamentar']))

,label,anoVotacao,data,idVotacao,tipoProposicao,parlamentar
346707,Avante,2009,2009-01-04 16:30:00,1-4-2009.16.30.3673,MPV,Vinicius Carvalho
346708,Avante,2009,2009-01-04 19:23:00,1-4-2009.19.23.3673,MPV,Vinicius Carvalho
346756,Avante,2009,2009-01-09 17:17:00,1-9-2009.17.17.3812,MPV,Vinicius Carvalho
346731,Avante,2009,2009-02-06 19:22:00,2-6-2009.19.22.3736,MPV,Vinicius Carvalho
346732,Avante,2009,2009-02-06 20:50:00,2-6-2009.20.50.3737,PEC,Vinicius Carvalho


,label,anoVotacao,parlamentar,periodo_mandato
346698,Avante,2009,Vinicius Carvalho,2007-2010


Possível problema com o idVotacao para identificar corretamente uma votação.
Foi usado date-timestamp como id, mas a miníma variação pode quebrar os votos de uma mesma proposição.

No dataframe abaixo pode-se observar que possivelmente a variação no timestamp separou os votos como fosse votações
distintas (index: [1-3]; [6-8]).

Por exemplo, em 2021, Avante tinha 8 parlamentares eleitos, considerando essa separação houve apenas um parlamentar votando (index de 1 a 3) sem a marcação dos outros 7 parlamentares. Isso ressalta mais um problema que é a falta de registro dos outros parlamentares (Ausente, Não, etc).

Agrupar por data também gerará problemas. Por exemplo, em 2009 (2007~2010), Avante teve 1 parlamentar eleito, se agruparmos por data isso gerará a informação de 2 votos (index 4 e 5), o que também é incorreto. 

In [22]:
tmp = basometro_df.groupby(['label','idVotacao','tipoProposicao','voto']).agg(count = ('voto','count')) \
         .reset_index() \
         .sort_values(by=['label','idVotacao','tipoProposicao'])
print(tmp.shape)
tmp.head(10)

(99075, 5)


,label,idVotacao,tipoProposicao,voto,count
0,Avante,1-10-2019.00.20.16943,PL,Não,2
1,Avante,1-12-2021.17.21.17189,PL,Sim,1
2,Avante,1-12-2021.17.39.17189,PL,Sim,1
3,Avante,1-12-2021.17.54.17189,PL,Sim,1
4,Avante,1-4-2009.16.30.3673,MPV,Não,1
5,Avante,1-4-2009.19.23.3673,MPV,Sim,1
6,Avante,1-7-2021.12.04.17120,PL,Sim,1
7,Avante,1-7-2021.12.24.17120,PL,Sim,1
8,Avante,1-7-2021.13.22.17120,PL,Sim,2
9,Avante,1-8-2017.18.14.16274,MPV,Não,1


In [24]:
print('Avante 2021:',basometro_df[(basometro_df['label']=='Avante') & (basometro_df['anoVotacao']==2021)]['parlamentar'].unique().tolist())
print('Avante 2019~2022:',basometro_df[(basometro_df['label']=='Avante') & (basometro_df['anoVotacao'].between(2019,2022))]['parlamentar'].unique().tolist())

Avante 2021: ['André Janones', 'Chiquinho Brazão', 'Leda Sadala']
Avante 2019~2022: ['Leda Sadala', 'André Janones', 'Chiquinho Brazão', 'Fernando Borja', 'Greyce Elias', 'Luis Tibé', 'Pastor Sargento Isidório', 'Tito']


Presença de parlamentares, mas sem registro ou baixo registro dos votos.

In [15]:
tmp_new_table = new_table[['label','idVotacao','ano','periodo_mandato','total_votos','qtd_parlamentares_partido']] \
                .copy() \
                .drop_duplicates()[new_table['total_votos']!=new_table['qtd_parlamentares_partido']]
tmp_new_table['diff_qtd_parlamentares_total_votos'] = tmp_new_table['qtd_parlamentares_partido'] - tmp_new_table['total_votos']

display(tmp_new_table.shape)
tmp_new_table.sort_values(by=['diff_qtd_parlamentares_total_votos'],ascending=False).head(5)

(68919, 7)

,label,idVotacao,ano,periodo_mandato,total_votos,qtd_parlamentares_partido,diff_qtd_parlamentares_total_votos
10042,MDB,17-3-2005.16.46.1918,2005,2003-2006,0,128,128
9021,MDB,1-6-2004.22.08.950,2004,2003-2006,0,128,128
9995,MDB,17-11-2004.18.58.1035,2004,2003-2006,1,128,127
11411,MDB,28-8-2003.17.30.824,2003,2003-2006,1,128,127
11380,MDB,28-5-2003.20.48.788,2003,2003-2006,2,128,126


In [44]:
basometro_df[(basometro_df['label']=='MDB') & (basometro_df['anoVotacao'].between(2003,2006))][['parlamentar']].nunique()

parlamentar    128
dtype: int64

In [66]:
df_gp = tmp_new_table.groupby(by=['label','periodo_mandato']).agg(avg=('diff_qtd_parlamentares_total_votos','mean')) \
                     .reset_index()
                     
df_gp['label_periodo'] = df_gp['periodo_mandato']+'_'+df_gp['label']
df_gp = df_gp.sort_values(by=['avg','label_periodo'],ascending=[False,False])
df_gp

,label,periodo_mandato,avg,label_periodo
15,MDB,2003-2006,78.744828,2003-2006_MDB
40,PL,2019-2022,68.693078,2019-2022_PL
10,DEM,2003-2006,67.220690,2003-2006_DEM
75,PSDB,2003-2006,66.098851,2003-2006_PSDB
17,MDB,2011-2014,62.165877,2011-2014_MDB
...,...,...,...,...
80,PSL,2003-2006,1.000000,2003-2006_PSL
21,PAN,2007-2010,0.903846,2007-2010_PAN
59,PRP,2019-2022,0.648649,2019-2022_PRP
9,DC,2015-2018,-0.471698,2015-2018_DC


In [69]:
fig = px.bar(df_gp[df_gp['avg']>=40], y='avg', x='label_periodo', text_auto='.2s',
            title="Diferença média entre qtd_parlamentares e total_votos por partido e período. <br>Filtro: avg>=40",
            width=1000,height=500)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [71]:
df_gp[df_gp['avg']>=40]['label'].value_counts()

MDB              4
PT               4
PSDB             3
PSD              3
DEM              2
PL               1
Progressistas    1
PSL              1
PSB              1
Name: label, dtype: int64

In [72]:
df_gp[df_gp['avg']>=40]['periodo_mandato'].value_counts()

2003-2006    6
2011-2014    4
2007-2010    4
2019-2022    3
2015-2018    3
Name: periodo_mandato, dtype: int64

Quatidade de votos maior que a quatidade de parlamentares?

In [18]:
tmp_new_table.sort_values(by=['total_votos'],ascending=False).head()

,label,idVotacao,ano,periodo_mandato,total_votos,qtd_parlamentares_partido,diff_qtd_parlamentares_total_votos
69832,União,23-2-2022.21.51.17212,2022,2019-2022,110,86,-24
69833,União,23-2-2022.23.38.17212,2022,2019-2022,108,86,-22
22209,PL,12-4-2022.16.57.17232,2022,2019-2022,108,104,-4
69831,União,23-2-2022.20.59.17212,2022,2019-2022,108,86,-22
69830,União,23-2-2022.20.41.17212,2022,2019-2022,107,86,-21


Avaliando problema de duplicatas nos votos no dataset que prejudica a contagem.

In [38]:
print('qtd parlamentares:',basometro_df[(basometro_df['label']=='União') & (basometro_df['anoVotacao'].between(2019,2022))]['parlamentar'].nunique())

mask = (basometro_df['idVotacao']=='23-2-2022.21.51.17212') & (basometro_df['label']=='União')
tmp = basometro_df[mask][['label','tipoProposicao','voto','parlamentar']]
tmp['rank_votos'] = tmp.apply(lambda x: tmp[tmp['parlamentar']==x['parlamentar']].shape[0],axis=1)
print(f"df.shape: {tmp.shape}")
print(tmp['voto'].value_counts())

tmp = tmp.drop_duplicates(subset=['voto','parlamentar'])
print(f"df.shape: {tmp.shape}")
print(tmp['voto'].value_counts())

tmp.sort_values(by=['rank_votos','parlamentar'],ascending=[False,False]).head(10)

qtd parlamentares: 86
df.shape: (110, 5)
Sim    55
Não    55
Name: voto, dtype: Int64
df.shape: (68, 5)
Não    36
Sim    32
Name: voto, dtype: Int64


,label,tipoProposicao,voto,parlamentar,rank_votos
27647,União,PL,Sim,Sóstenes Cavalcante,3
395829,União,PL,Não,Pedro Lupion,3
342308,União,PL,Sim,Nelson Barbudo,3
274898,União,PL,Não,Márcio Labre,3
98147,União,PL,Sim,Léo Motta,3
68043,União,PL,Não,Jose Mario Schreiner,3
93498,União,PL,Sim,Filipe Barros,3
92886,União,PL,Sim,Eduardo Bolsonaro,3
92236,União,PL,Sim,Coronel Tadeu,3
103582,União,PL,Sim,Vitor Hugo,2


In [40]:
print('qtd parlamentares:',basometro_df[(basometro_df['label']=='PL') & (basometro_df['anoVotacao'].between(2019,2022))]['parlamentar'].nunique())

mask = (basometro_df['idVotacao']=='12-4-2022.16.57.17232') & (basometro_df['label']=='PL')
tmp = basometro_df[mask][['label','tipoProposicao','voto','parlamentar']]
tmp['rank_votos'] = tmp.apply(lambda x: tmp[tmp['parlamentar']==x['parlamentar']].shape[0],axis=1)
print(f"df.shape: {tmp.shape}")
print(tmp['voto'].value_counts())

tmp = tmp.drop_duplicates(subset=['voto','parlamentar'])
print(f"df.shape: {tmp.shape}")
print(tmp['voto'].value_counts())

tmp.sort_values(by=['rank_votos','parlamentar'],ascending=[False,False]).head(10)

qtd parlamentares: 104
df.shape: (108, 5)
Sim    108
Name: voto, dtype: Int64
df.shape: (74, 5)
Sim    74
Name: voto, dtype: Int64


,label,tipoProposicao,voto,parlamentar,rank_votos
27704,PL,PL,Sim,Sóstenes Cavalcante,3
342395,PL,PL,Sim,Nelson Barbudo,3
274993,PL,PL,Sim,Márcio Labre,3
93573,PL,PL,Sim,Filipe Barros,3
92952,PL,PL,Sim,Eduardo Bolsonaro,3
92317,PL,PL,Sim,Coronel Tadeu,3
103673,PL,PL,Sim,Vitor Hugo,2
121383,PL,PL,Sim,Silvia Cristina,2
343849,PL,PL,Sim,Sanderson,2
102198,PL,PL,Sim,Rosana Valle,2


Tudo indica ser um problema de registros duplicados. A análise tem diferentes perspectivas, como agrupados e pivôs mudando a granularidade. Além disso, a extração pode ter gerado isso uma vez que uma votação pode ter uma jornada dentro da câmar - o que possivelmente explique a duplicata. A duplicata pode aparecer pois atualmente olhamos o todo, desconsiderando a possivel jornada de uma dada votação. Algumas regras tiravam as duplicatas, mas nas outras que isso não acontecia fez aparecer as divergências.

Com a identificação acima, fazendo este tratamente possivelmente solucionar este problema. Contudo, podemos notar que os valores totais ainda não se igualam, ou seja, ainda temos casos de registros que não foram computados. Talvez isso possa ficar no novo atributo 'ausente_calculado' após a correção.